In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score



IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
DATA_ROOT = 'C:/Users/chira/Desktop/VideoAnalysisTry/screenshots/Cashier'
IMG_SIZE = 64  # Reduced size for SVM performance
BATCH_SIZE = 32

data_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
])


print("Loading datasets...")
train_dataset = datasets.ImageFolder(root=DATA_ROOT + '/Train', transform=data_transforms)
test_dataset = datasets.ImageFolder(root=DATA_ROOT + '/Test', transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

def extract_flattened_data(loader):
    features = []
    labels = []
    print(f"Extracting features from {len(loader.dataset)} images...")
    
    for images, target_labels in loader:
        # Reshape from [Batch, 3, 64, 64] to [Batch, 12288]
        flattened = images.view(images.size(0), -1).numpy()
        features.append(flattened)
        labels.append(target_labels.numpy())
        
    return np.vstack(features), np.concatenate(labels)


X_train, y_train = extract_flattened_data(train_loader)
X_test, y_test = extract_flattened_data(test_loader)


print("Training SVM model... (this may take a minute)")
clf = svm.SVC(kernel='linear', C=1.0, verbose=True)
clf.fit(X_train, y_train)


print("\n--- Evaluation Results ---")
y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\nDetailed Report:")
print(classification_report(y_test, y_pred, target_names=train_dataset.classes))

